# YOLO DenseNet Hybrid pipeline
## hybrid_pipeline.py

In [ ]:
from ultralytics import YOLO
import torch
import cv2
import matplotlib.pyplot as plt
import os
import glob
import logging
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import Adam
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
import numpy as np
from collections import Counter
import random
%matplotlib inline

# Logging config
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("HybridPipeline")

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 1. Train YOLO model

Tätä ei tehdä vaan ladataan mika_best malli

In [ ]:
"""
def train_yolo():
    print("\n🔧 Koulutetaan YOLO-mallia...")
    model = YOLO('yolo11s.pt')
    model.train(
        data='config/materials.yaml',
        epochs=40,
        imgsz=768,
        batch=16,
        patience=20,
        cache=False,
        project='runs/train',
        name='materials_test_direct',
        exist_ok=True,
        lr0=0.003,
        momentum=0.937,
        weight_decay=0.0001,
        warmup_epochs=5.0,
        warmup_momentum=0.8,
        box=0.05,
        cls=0.5,
        dfl=1.5,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        translate=0.1,
        scale=0.8,
        perspective=0.002,
        fliplr=0.5,
        mosaic=0.2,
        mixup=0.1,
        copy_paste=0.05
    )
    torch.cuda.empty_cache()
    print("✅ YOLO-koulutus valmis.")
"""

## 2. Validate YOLO model

mika_best validointi. Klikkaa pois, jos ei tarvetta

In [ ]:
def validate_yolo():
    print("\n🔍 Validoidaan YOLO-malli...")
    model = YOLO('paras_yolo_malli/mika_best.pt') 
    results = model.val(augment=True)

    print(f"\nmAP@0.5: {results.box.map50:.4f}")
    print(f"mAP@0.5:0.95: {results.box.map:.4f}\n")

    ap_list = results.box.ap
    p_list = results.box.p
    r_list = results.box.r
    class_names = list(results.names.values())

    for i in range(len(ap_list)):
        print(f"Class '{class_names[i]}': Precision = {p_list[i]:.4f}, Recall = {r_list[i]:.4f}, AP = {ap_list[i]:.4f}")

    print("✅ YOLO-validointi valmis.")


## 3. Generate cropped images

kuvien croppaus

In [ ]:
def generate_cropped_images(image_root, label_root, output_root):
    print(f"\n✂️  Rajataan kuvia hakemistoista: {image_root} → {output_root}")
    os.makedirs(output_root, exist_ok=True)
    image_paths = sorted(glob.glob(os.path.join(image_root, '*.jpg')) + glob.glob(os.path.join(image_root, '*.png')))
    for img_path in image_paths:
        name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(label_root, f"{name}.txt")
        if not os.path.exists(label_path): continue
        img = cv2.imread(img_path)
        if img is None: continue
        h, w = img.shape[:2]
        with open(label_path, 'r') as f:
            for idx, line in enumerate(f):
                class_id, xc, yc, bw, bh = map(float, line.strip().split())
                x1 = max(0, int((xc - bw / 2) * w))
                y1 = max(0, int((yc - bh / 2) * h))
                x2 = min(w, int((xc + bw / 2) * w))
                y2 = min(h, int((yc + bh / 2) * h))
                cropped = img[y1:y2, x1:x2]
                if cropped.size == 0: continue
                out_path = os.path.join(output_root, str(int(class_id)), f"{name}_{idx}.jpg")
                os.makedirs(os.path.dirname(out_path), exist_ok=True)
                cv2.imwrite(out_path, cropped)
    print("✅ Kuvien rajaaminen valmis.")

## 4. Train DenseNet

Tallentaa tuloksen "densenet_materials_best.pth"


In [ ]:
def train_densenet():
    print("\n🏋️‍♂️ Koulutetaan DenseNet-malli...")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    train_data = datasets.ImageFolder('datasets/materials/cropped/train', transform=transform)
    val_data = datasets.ImageFolder('datasets/materials/cropped/validation', transform=transform)

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

    model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
    model.classifier = nn.Linear(model.classifier.in_features, len(train_data.classes))
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-4)

    best_val_acc = 0.0
    train_losses, val_accuracies = [], []

    for epoch in range(25):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_losses.append(running_loss / len(train_loader))
        train_acc = 100 * correct / total

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1:02d}: Train Acc = {train_acc:.2f}%, Val Acc = {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'densenet_materials_best.pth')

    print("✅ DenseNet-koulutus valmis.")

## 5. Evaluate Densenet

In [ ]:
def evaluate_densenet():
    print("\n🔍 Arvioidaan DenseNet-mallia validointidatalla...")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    val_data = datasets.ImageFolder('datasets/materials/cropped/validation', transform=transform)
    val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

    model = models.densenet121(weights=None)
    model.classifier = nn.Linear(model.classifier.in_features, len(val_data.classes))
    model.load_state_dict(torch.load("densenet_materials_best.pth", map_location=device))
    model.to(device)
    model.eval()

    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    print("\n✅ DenseNet tarkkuusvalidaatio:")
    print(classification_report(y_true, y_pred, target_names=val_data.classes, zero_division=0))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


## 6. Evaluate hybrid model

In [ ]:
def evaluate_hybrid():
    print("\n🔍 Arvioidaan hybridimallia...")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    class_names = datasets.ImageFolder("datasets/materials/cropped/train", transform=transform).classes
    densenet_model = models.densenet121(weights=None)
    densenet_model.classifier = nn.Linear(densenet_model.classifier.in_features, len(class_names))
    densenet_model.load_state_dict(torch.load("densenet_materials_best.pth", map_location=device))
    densenet_model.to(device)
    densenet_model.eval()

    yolo_model = YOLO("paras_yolo_malli/mika_best.pt")

    image_paths = glob.glob("datasets/materials/images/validation/*.jpg") + \
                  glob.glob("datasets/materials/images/validation/*.png")

    results_counter = Counter()

    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is None: continue
        results = yolo_model(img_path)[0]
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            yolo_cls = int(box.cls[0])
            yolo_conf = float(box.conf[0])
            crop = img[y1:y2, x1:x2]
            if crop.shape[0] == 0 or crop.shape[1] == 0:
                continue
            pil_img = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
            input_tensor = transform(pil_img).unsqueeze(0).to(device)
            with torch.no_grad():
                output = densenet_model(input_tensor)
                densenet_cls = output.argmax(dim=1).item()
                densenet_conf = torch.softmax(output, dim=1)[0][densenet_cls].item()

            if yolo_conf >= 0.75:
                final_cls = yolo_cls
                source = "YOLO"
            else:
                final_cls = densenet_cls
                source = "DenseNet"

            results_counter[(class_names[final_cls], source)] += 1

    print("\n📊 Hybridiluokituksen tulokset:")
    for (cls_name, source), count in sorted(results_counter.items()):
        print(f"{cls_name:25s} [{source}]: {count} kpl")
    print("✅ Hybridimallin arviointi valmis.")

In [ ]:
def run_random_examples():
    print("\n🎲 Ajetaan 3 satunnaista esimerkkikuvaa hybridimallilla...")
    all_images = glob.glob("datasets/materials/images/validation/*.jpg") + \
                 glob.glob("datasets/materials/images/validation/*.png")
    used_images = ["datasets/materials/images/validation/174114_2.jpg"]
    remaining_images = list(set(all_images) - set(used_images))
    random_images = random.sample(remaining_images, 3)
    for path in random_images:
        show_yolo_vs_densenet(path)

In [ ]:
def in_notebook():
    try:
        shell = get_ipython().__class__.__name__
        return shell == 'ZMQInteractiveShell'
    except NameError:
        return False

if __name__ == "__main__":
    if in_notebook():
        print("📘 Ajetaan Jupyterissa – ohitetaan komentoriviargumentit.")
    else:
        import argparse

        parser = argparse.ArgumentParser(description="YOLO + DenseNet hybrid pipeline")
        #parser.add_argument('--train_yolo', action='store_true', help='Train YOLO model')
        parser.add_argument('--validate_yolo', action='store_true', help='Validate YOLO model')
        parser.add_argument('--generate_crops', action='store_true', help='Generate cropped images from YOLO labels')
        parser.add_argument('--train_densenet', action='store_true', help='Train DenseNet model')
        parser.add_argument('--evaluate_hybrid', action='store_true', help='Evaluate hybrid YOLO + DenseNet model')
        parser.add_argument('--evaluate_densenet', action='store_true', help='Evaluate DenseNet model performance on validation set')
print(type(batch))
        args = parser.parse_args()

        #if args.train_yolo:
        #    train_yolo()

        if args.validate_yolo:
            validate_yolo()

        if args.generate_crops:
            generate_cropped_images('datasets/materials/images/train', 'datasets/materials/labels/train', 'datasets/materials/cropped/train')
            generate_cropped_images('datasets/materials/images/validation', 'datasets/materials/labels/validation', 'datasets/materials/cropped/validation')

        if args.train_densenet:
            train_densenet()

        if args.evaluate_hybrid:
            evaluate_hybrid()

        if args.evaluate_densenet:
            evaluate_densenet()

        if args.show_examples:
            run_random_examples()


In [ ]:
def list_files(directory):
  try:
    filenames = os.listdir(directory)
    return filenames
  except FileNotFoundError:
    return []

def compare_dense_models():
    models = list_files("./paras_dense_malli")
    for model in models:
        evaluate_chosen_densenet(model)
    


## 7. Kutsu

In [ ]:
#train_yolo()
validate_yolo()
generate_cropped_images('datasets/materials/images/train', 'datasets/materials/labels/train', 'datasets/materials/cropped/train')
generate_cropped_images('datasets/materials/images/validation', 'datasets/materials/labels/validation', 'datasets/materials/cropped/validation')
train_densenet()
evaluate_densenet()
evaluate_hybrid()



In [ ]:
def evaluate_chosen_densenet(model_name, model_architecture=None):
    print(f"\n🔍 Arvioidaan DenseNet-mallia {model_name} validointidatalla...")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    val_data = datasets.ImageFolder('datasets/materials/cropped/validation', transform=transform)
    val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

    if "d169" in model_name:
        model = models.densenet169(weights=None)
    else:
        model = models.densenet121(weights=None)
    model.classifier = nn.Linear(model.classifier.in_features, len(val_data.classes))
    model.load_state_dict(torch.load(f"./paras_dense_malli/{model_name}", map_location=device))
    model.to(device)
    model.eval()

    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    print("\n✅ DenseNet tarkkuusvalidaatio:")
    print(classification_report(y_true, y_pred, target_names=val_data.classes, zero_division=0))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


In [ ]:
generate_cropped_images('datasets/materials/images/validation', 'datasets/materials/labels/validation', 'datasets/materials/cropped/validation')
compare_dense_models()

In [ ]:
def show_yolo_vs_densenet(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    class_names = datasets.ImageFolder("datasets/materials/cropped/train", transform=transform).classes

    yolo_model = YOLO("paras_yolo_malli/mika_best.pt")
    densenet_model = models.densenet121(weights=None)
    densenet_model.classifier = nn.Linear(densenet_model.classifier.in_features, len(class_names))
    densenet_model.load_state_dict(torch.load("densenet_materials_best.pth", map_location=device))
    densenet_model.to(device)
    densenet_model.eval()

    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    yolo_img, densenet_img, hybrid_img = img.copy(), img.copy(), img.copy()

    results = yolo_model.predict(image_path, verbose=False)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        yolo_cls = int(box.cls[0])
        yolo_conf = float(box.conf[0])
        label = f"{class_names[yolo_cls]} ({yolo_conf*100:.1f}%)"

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale, thickness = 1.5, 4
        (text_w, text_h), _ = cv2.getTextSize(label, font, font_scale, thickness)
        text_x = x1 + (x2 - x1 - text_w) // 2
        text_y = max(y1 - 10, text_h + 10)

        cv2.rectangle(yolo_img, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(yolo_img, label, (text_x, text_y), font, font_scale, (0, 255, 255), thickness)

        crop = densenet_img[y1:y2, x1:x2]
        if crop.size == 0:
            continue

        pil_img = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        input_tensor = transform(pil_img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = densenet_model(input_tensor)
            pred_cls = output.argmax(dim=1).item()
            dn_conf = torch.softmax(output, dim=1)[0][pred_cls].item()

        dn_label = f"{class_names[pred_cls]} ({dn_conf*100:.1f}%)"
        cv2.rectangle(densenet_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(densenet_img, dn_label, (text_x, text_y), font, font_scale, (0, 255, 0), thickness)

        if yolo_conf >= 0.75:
            final_cls, final_conf, source, color = yolo_cls, yolo_conf, "YOLO", (0, 255, 255)
        else:
            final_cls, final_conf, source, color = pred_cls, dn_conf, "DenseNet", (0, 255, 0)

        hybrid_label = f"{class_names[final_cls]} ({final_conf*100:.1f}%) [{source}]"
        cv2.rectangle(hybrid_img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(hybrid_img, hybrid_label, (text_x, text_y), font, font_scale, color, thickness)

    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(yolo_img, cv2.COLOR_BGR2RGB))
    plt.title("YOLO-luokitus")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(cv2.cvtColor(densenet_img, cv2.COLOR_BGR2RGB))
    plt.title("DenseNet-luokitus")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(hybrid_img, cv2.COLOR_BGR2RGB))
    plt.title("Hybridiluokitus")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

    print("\n📘 Luokkakoodien selitykset:")
    for idx, name in enumerate(class_names):
        print(f"{idx} = {name}")

In [ ]:
show_yolo_vs_densenet("datasets/materials/images/validation/176137_1.jpg")


In [ ]:
# Poistaa jo käytetyt ja valitsee kolme uutta kuvaa
run_random_examples()


## Tämä on itse kuvattu ja ladattu kuva

In [ ]:
show_yolo_vs_densenet("datasets/materials/images/testikuva.jpg")